In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import folium
from folium import Map, CircleMarker, Vega, Popup,IFrame
import base64
from rpy2.robjects import r, numpy2ri
from rpy2.robjects import numpy2ri
numpy2ri.activate()
from rpy2.robjects.packages import importr
sinkr = importr('sinkr')

In [2]:
data=pd.read_csv('v100_hr_20200101-0930_cut_utf8.csv')

In [3]:
###
#X
###
u=np.zeros([244,8520])

for i in range (0,1704):
    u[0][i]=np.nan

for i in range (0,243):
    a=np.array(data['cal_PMf'][i*6816:(i*6816)+24])
    for j in range (1,71):
        b=np.array(data['cal_PMf'][(i*6816)+(96*j):(i*6816)+(96*j)+24])
        a=np.append(a, b)
    for k in range(1,1704):
        if a[k]=='\\N' :
            a[k]=np.nan
    for l in range (0,1704):
        u[i+1][l]=a[l]

for i in range (0,244):
    a=np.array(data['cal_PMf'][6816*i:6816*i+6816])
    for j in range(0,6816):
        if a[j]=='\\N' :
            a[j]=np.nan
    for l in range (1704,8520):
        u[i][l]=a[l-1704]
###
#Y
###
v=np.zeros([244,6816])
for i in range (0,244):
    a=np.array(data['obs_PMf'][6816*i:6816*i+6816])
    for j in range(0,6816):
        if a[j]=='\\N' :
            a[j]=np.nan
    v[i]=a.T
###
#station
###
station=np.zeros([71,2])
for i in range (0,71):
    station[i][0]=np.array(data['lat'][96*i])
    station[i][1]=np.array(data['lon'][96*i])
# u=u.T
# print(station)

#print(np.shape(station))
print(np.shape(u))
print(np.shape(v))

(244, 8520)
(244, 6816)


# DINEOF (like FPCA)

In [4]:
XRestruct=sinkr.dineof(u)
YRestruct=sinkr.dineof(v)

[1]
 "1 EOF ; RMS = 10.61889475"


[1]
 "1 EOF ; RMS = 10.60519598"


[1]
 "1 EOF ; RMS = 10.60425216"


[1]
 "1 EOF ; RMS = 10.6040627"


[1]
 "1 EOF ; RMS = 10.60402334"


[1]
 "1 EOF ; RMS = 10.60401644"


[1]
 "2 EOF ; RMS = 9.80634388"


[1]
 "2 EOF ; RMS = 9.80781605"


[1]
 "3 EOF ; RMS = 9.50488722"


[1]
 "3 EOF ; RMS = 9.50863472"


[1]
 "4 EOF ; RMS = 9.28119766"


[1]
 "4 EOF ; RMS = 9.28633537"


[1]
 "5 EOF ; RMS = 9.04375374"


[1]
 "5 EOF ; RMS = 9.0448816"


[1]
 "6 EOF ; RMS = 8.91535894"


[1]
 "6 EOF ; RMS = 8.93557899"


[1]
 "7 EOF ; RMS = 8.83382644"


[1]
 "7 EOF ; RMS = 8.84005272"


[1]
 "8 EOF ; RMS = 8.77853642"


[1]
 "8 EOF ; RMS = 8.7991183"


[1]
 "9 EOF ; RMS = 8.77013199"


[1]
 "9 EOF ; RMS = 8.79325409"


[1]
 "10 EOF ; RMS = 8.68145786"


[1]
 "10 EOF ; RMS = 8.68332213"


[1]
 "11 EOF ; RMS = 8.58788348"


[1]
 "11 EOF ; RMS = 8.59030811"


[1]
 "12 EOF ; RMS = 8.51970472"


[1]
 "12 EOF ; RMS = 8.52877677"


[1]
 "13 EOF ; RMS = 8.53297894"


[1]


# Restruct Function

In [5]:
XRestruct_Fun=np.array(XRestruct[0])
YRestruct_Fun=np.array(YRestruct[0])
print(np.shape(XRestruct_Fun))
print(np.shape(YRestruct_Fun))

(244, 8520)
(244, 6816)


In [6]:
XCov_fun=np.cov(XRestruct_Fun.T)
print(np.shape(XCov_fun))

(8520, 8520)


In [7]:
XMean_fun=np.zeros(8520)
for i in range(0,8520):
    for j in range(0,244):
        XMean_fun[i]=XMean_fun[i]+(XRestruct_Fun[j][i]/244)
YMean_fun=np.zeros(6816)
for i in range(0,6816):
    for j in range(0,244):
        YMean_fun[i]=YMean_fun[i]+(YRestruct_Fun[j][i]/244)
print(np.shape(XMean_fun))
print(np.shape(YMean_fun))

(8520,)
(6816,)


In [8]:
Xhat=XRestruct_Fun
Yhat=YRestruct_Fun
Xbar=XMean_fun
Ybar=YMean_fun
print('Xhat')
#print(Xhat)
print(np.shape(Xhat))
print('Yhat')
#print(Yhat)
print(np.shape(Yhat))
print('Xbar')
#print(Xbar)
print(np.shape(Xbar))
print('Ybar')
#print(Ybar)
print(np.shape(Ybar))

Xhat
(244, 8520)
Yhat
(244, 6816)
Xbar
(8520,)
Ybar
(6816,)


# Regression

In [9]:
nXCov_fun=np.zeros([6816,6816])
for i in range (1704,8520):
    for j in range (1704,8520):
        nXCov_fun[i-1704][j-1704]=XCov_fun[i][j]

G_xx=nXCov_fun
G_xxt=np.zeros(6816)
for i in range(0,6816):
     G_xxt[i]=G_xx[i][i]
print(G_xxt)
print(np.shape(G_xxt))

G_xy=0
# for i in range (0,244):
#     G_xyi=(Xhat[i]-Xbar)*(Yhat[i]-Ybar)
#     G_xy=G_xy+G_xyi
# G_xy=G_xy/243
G_xy=np.cov(Xhat.T,Yhat.T)
print(G_xy)
print(np.shape(G_xy))

[ 68.15802914  63.24933945 116.31025274 ...  98.99547747 118.30559943
 139.82751368]
(6816,)
[[ 88.49661921  74.03814033  81.13381645 ...  -2.90060036  -3.64480207
   -2.98852343]
 [ 74.03814033  85.24888308  96.4424401  ...  -5.79922629  -4.07075548
   -4.08978987]
 [ 81.13381645  96.4424401  150.39634966 ...  -9.3419944   -4.9008665
   -6.056324  ]
 ...
 [ -2.90060036  -5.79922629  -9.3419944  ... 190.37186203 181.70854018
  190.28440021]
 [ -3.64480207  -4.07075548  -4.9008665  ... 181.70854018 197.15749331
  203.67973024]
 [ -2.98852343  -4.08978987  -6.056324   ... 190.28440021 203.67973024
  233.08514237]]
(15336, 15336)


In [9]:
# B_1=G_xy/G_xxt
# B_0=YMean_fun-(B_1*XMean_fun)
# B_1const=0
# for i in range (0,6816):
#     B_1const=B_1const+B_1[i]
# B_1const=B_1const/6816

# x1=np.linalg.inv(np.dot(Xhat.T,Xhat))

x1=np.linalg.solve(np.dot(Xhat.T,Xhat),np.identity(np.shape(np.dot(Xhat.T,Xhat))[0]))
x2=np.dot(Xhat.T,Yhat)
B_1=np.dot(x1,x2)
print('x1')
print(x1)
print(np.shape(x1))
# for i in range(0,244):
#     for j in range(0,6816):
#         B_1[i][j+1704]=x2[i][j]

B_0=YMean_fun-np.dot(XMean_fun,B_1)

print('B_1')
#print(B_1)
print(np.shape(B_1))
print('B_0')
#print(B_0)
print(np.shape(B_0))

x1
[[-1.48016134e+09 -6.27109754e+07 -3.68141754e+09 ...  2.16148474e+09
   1.04511443e+09 -8.15689562e+08]
 [ 8.06310094e+08  2.95046387e+08  2.41444898e+09 ... -3.02360718e+08
  -3.50134298e+08  4.92817794e+07]
 [-9.50125276e+08 -5.02026748e+08 -2.43820858e+09 ... -7.09420085e+08
   5.18733680e+08 -1.61909269e+08]
 ...
 [-3.07531648e+07 -5.53163877e+06 -2.25048962e+08 ... -2.62553483e+08
  -2.09953788e+07 -1.70221523e+07]
 [-1.32045521e+09 -4.50183845e+08 -2.65108605e+09 ...  1.73359268e+08
   4.85499879e+08 -6.13056107e+07]
 [-3.50461103e+08 -6.79416705e+07 -8.33566652e+08 ...  7.00405915e+08
   2.70802740e+08 -4.17074175e+07]]
(8520, 8520)
B_1
(8520, 6816)
B_0
(6816,)


In [11]:
before_fx=np.zeros([244,8520])
for i in range (0,1704):
    before_fx[0][i]=np.nan

for i in range (0,243):
    a=np.array(data['cal_PMf'][i*6816:(i*6816)+24])
    for j in range (1,71):
        b=np.array(data['cal_PMf'][(i*6816)+(96*j):(i*6816)+(96*j)+24])
        a=np.append(a, b)
    for k in range(1,1704):
        if a[k]=='\\N' :
            a[k]=np.nan
    for l in range (0,1704):
        before_fx[i+1][l]=a[l]

for i in range (0,244):
    a=np.array(data['cal_PMf'][6816*i:6816*i+6816])
    for j in range(0,6816):
        if a[j]=='\\N' :
            a[j]=np.nan
    for l in range (1704,8520):
        before_fx[i][l]=a[l-1704]
print(np.shape(before_fx))
###
#Y
###
# v=np.zeros([244,6816])
# for i in range (0,244):
#     a=np.array(data['obs_PMf'][6816*i:6816*i+6816])
#     for j in range(0,6816):
#         if a[j]=='\\N' :
#             a[j]=np.nan
#     v[i]=a.T

(244, 8520)


In [12]:
after_fx_r=sinkr.dineof(before_fx)
after_fx=np.array(after_fx_r[0])
print(np.shape(after_fx))

[1]
 "1 EOF ; RMS = 10.78071353"


[1]
 "1 EOF ; RMS = 10.76957098"


[1]
 "1 EOF ; RMS = 10.77054658"


[1]
 "2 EOF ; RMS = 10.03708963"


[1]
 "2 EOF ; RMS = 10.03907425"


[1]
 "3 EOF ; RMS = 9.68238741"


[1]
 "3 EOF ; RMS = 9.68335862"


[1]
 "4 EOF ; RMS = 9.41740715"


[1]
 "4 EOF ; RMS = 9.41994826"


[1]
 "5 EOF ; RMS = 9.28617544"


[1]
 "5 EOF ; RMS = 9.29534163"


[1]
 "6 EOF ; RMS = 9.14215855"


[1]
 "6 EOF ; RMS = 9.15145314"


[1]
 "7 EOF ; RMS = 9.02022078"


[1]
 "7 EOF ; RMS = 9.02584545"


[1]
 "8 EOF ; RMS = 8.94753061"


[1]
 "8 EOF ; RMS = 8.95849754"


[1]
 "9 EOF ; RMS = 8.88352869"


[1]
 "9 EOF ; RMS = 8.89945239"


[1]
 "10 EOF ; RMS = 8.87562633"


[1]
 "10 EOF ; RMS = 8.89523648"


[1]
 "11 EOF ; RMS = 8.85210289"


[1]
 "11 EOF ; RMS = 8.86421636"


[1]
 "12 EOF ; RMS = 8.82016984"


[1]
 "12 EOF ; RMS = 8.82636516"


[1]
 "13 EOF ; RMS = 8.7783425"


[1]
 "13 EOF ; RMS = 8.79105778"


[1]
 "14 EOF ; RMS = 8.74336352"


[1]
 "14 EOF ; RMS = 8.75782331"




In [13]:
fit_fx=B_0+np.dot(after_fx,B_1)
print(np.shape(fit_fx))

(244, 6816)


In [14]:
old_x=np.zeros([244,6816])
old_y=np.zeros([244,6816])
#data_cal
for i in range (0,244):
    a=np.array(data['cal_PMf'][6816*i:6816*i+6816])
    for j in range(0,6816):
        if a[j]=='\\N' :
            a[j]=0
    old_x[i]=a
#data_obs
for i in range (0,244):
    a=np.array(data['obs_PMf'][6816*i:6816*i+6816])
    for j in range(0,6816):
        if a[j]=='\\N' :
            a[j]=0
    old_y[i]=a
#Delete missing data
for i in range(0,244):
    for j in range(6816):
        if old_x[i][j]==0:
            old_y[i][j]=0
            fit_fx[i][j]=0
        if old_y[i][j]==0:
            old_x[i][j]=0
            fit_fx[i][j]=0
print(np.shape(old_x))
print(np.shape(old_y))
print(np.shape(fit_fx))

(244, 6816)
(244, 6816)
(244, 6816)


In [15]:
before_error1=0
after_error1=0
for i in range (0,244):
    for j in range(0,6816):
        before_error1=before_error1+abs(old_x[i][j]-old_y[i][j])
        after_error1=after_error1+abs(fit_fx[i][j]-old_y[i][j])
before_error1=before_error1/244/6816
after_error1=after_error1/244/6816
print("January to August (training data set) before_error: %f" %before_error1)
print("January to August (training data set) after_error: %f" %after_error1)

January to August (training data set) before_error: 7.961740
January to August (training data set) after_error: 16159.820656
